<a href="https://colab.research.google.com/github/teacherSsamko/DL-study/blob/main/w4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [4주차] 심화과제: 수능 국어 문제 GPT-4로 풀어보기

In [1]:
!pip install openai

## [MY CODE] 수능 문제 데이터 불러오기

In [2]:
import json
from pathlib import Path

data_file = Path('2023_11_KICE.json')

with data_file.open('r') as f:
  data = json.load(f)

print(data[0])

{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통

## [LOG] 데이터 분석

- 하나의 보기에 문제 그룹이 있고, question_plus라는 추가 보기가 있는 문제도 존재

## [MY CODE] prediction 함수

In [30]:
from openai import OpenAI

def prediction(
    client: OpenAI,
    paragraph: str,
    question: str,
    question_plus: str,
    choices: list[str]
  ) -> str:
  choices_str = '\n'.join([f'{i+1}. {choice}' for i, choice in enumerate(choices)])
  prompt = f"""
  <지문>
  {paragraph}
  <문제>
  {question}
  <보기>
  {question_plus}
  선택지:
  {choices}

  정답:

  let's think step by step
  """
  response = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": """
너는 수능 문제를 푸는 똑똑한 학생이야. 대답은 한글로 해. 지문을 읽고, 문제에 대한 답을 선택지에서 골라야 해.
만약 <보기>에 내용이 있으면, <보기>도 참고해서 답을 골라야 해.
선택지는 1번~5번까지 주어지는데, 반드시 모든 선택지를 다 읽고, 정답을 하나만 골라야 해.
""",
          },
          {
          "role": "user",
          "content": prompt,
        },
      ],
      model="gpt-4o",
  )
  content = response.choices[0].message.content
  return content


## [MY CODE] parse_answer 함수

- gpt응답에서 정답만을 추출하는 함수
- gpt응답이 일정한 형태로 오지 않지만 마지막 문장에 숫자로 답을 표현하는 경우가 일반적이기 때문에, 마지막 줄의 숫자를 추출
- prompt에서 답변 형태를 제안할 경우 성능(정확도)이 떨어지는 것이 확인되어서 답변 형태를 제안하지 않음.


In [26]:
import re

def parse_answer(answer: str) -> int:
  last_line = answer.split("\n")[-1]
  pattern = r"(\d+)"
  match = re.search(pattern, last_line)
  if match:
    return int(match.group(1))

  return 0

## [MY CODE] 수능 문제 풀이

- gpt응답에서 정답을 잘 parsing하는지 확인하기 위해 gpt answer 로그 출력
- 오답인 경우 gpt가 어떤 응답을 했는지 확인하기 위해 gpt응답 출력


In [31]:
# Need to set 'OPENAI_API_KEY' as an environment variable to initiate client
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)
score = 0

for q_group in data:
  for q in q_group['problems']:
    res = prediction(
        client,
        q_group['paragraph'],
        q['question'],
        q.get('question_plus', ""),
        q['choices'],
    )
    gpt_answer = parse_answer(res)
    print(f"gpt answer: {gpt_answer}")
    if gpt_answer == q['answer']:
      score += q['score']
    else:
      print(res)

print(f"score: {score}")


gpt answer: 4
gpt answer: 5
gpt answer: 1
gpt answer: 4
gpt answer: 5
gpt answer: 3
gpt answer: 2
gpt answer: 5
gpt answer: 2
gpt answer: 1
주어진 지문에서는 불확정 개념이 법령의 조문에 어떻게 사용되는지 설명하고 있습니다. 주요 내용은 다음과 같습니다:

1. 법령의 조문은 요건과 효과로 구성된 조건문 형태로 규정됩니다.
2. 그렇지만 요건이나 효과가 항상 일의적이지는 않습니다. 이는 불확정 개념이 사용될 수 있기 때문입니다.
3. 불확정 개념이 사용된 법령은 구체적 상황을 고려해야 그 의미가 파악됩니다.
4. 법원은 불확정 개념이 사용된 민법 조항에 대해서 재량을 가지고 판단할 수 있습니다.
5. 불확정 개념은 행정 법령에도 사용되며, 이러한 경우 행정청은 재량을 행사할 수 있습니다.
6. 행정청은 재량 준칙을 정할 수 있지만, 이 준칙대로의 재량 행사가 법령 위반은 아닙니다.
7. 행정청은 평등 원칙을 따라 동일한 요건이 충족되면 같은 내용의 행정 작용을 해야 합니다.

선택지 분석:

1. "법령의 요건과 효과에는 모두 불확정 개념이 사용될 수 있다." - 이는 틀립니다. 모든 요건과 효과에 불확정 개념이 사용되는 것이 아니며, 불확정 개념은 상황에 따라 사용될 수 있다고 지문은 설명하고 있습니다.

2. "법원은 불확정 개념이 사용된 법령을 적용할 때 재량을 행사할 수 있다." - 이는 맞습니다. 지문에 이에 대한 설명이 있습니다.

3. "불확정 개념이 사용된 법령의 진정한 의미를 이해하려면 구체적 상황을 고려해야 한다." - 이는 맞습니다. 지문에서 언급된 내용입니다.

4. "불확정 개념이 사용된 행정 법령에 근거한 행정 작용은 재량 행위인 경우보다 기속 행위인 경우가 많다." - 이는 틀립니다. 불확정 개념이 사용된 경우 재량 행위인 경우가 많다고 지문은 설명하고 있습니다.

5. "불확정 개념은 행정청이 행하는 법 집행 작용을 규율하는

## [LOG] 85점

- 마지막 오답을 제외하면, 정답 parsing은 전반적으로 제대로 이루어진 것으로 보입니다.
- KICE slayer의 결과표와 비교해봐도 준수한 성적